### 1. 각종 라이브러리 및 모델 임포트 


In [1]:
!pip3 install lxml

     ---------------------------------------- 3.8/3.8 MB 2.6 MB/s eta 0:00:00


In [35]:
!pip install -U deep-translator 

In [3]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 55.1/55.1 kB 2.8 MB/s eta 0:00:00
     -------------------------------------- 133.4/133.4 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 58.8/58.8 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 65.0/65.0 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 53.6/53.6 kB 2.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17411 sha256=3b573ab7ba4e805621b696a85dd19b06cbea601b16731e8a5c459aa8c3e8c346
  Stored in directory: c:\users\junho\appdata\local\pip\cache\wheels\54\ca\27\562b6eac3a495887e4b44bac3a1efe925fa603d085ba89a21d
Successfully built googletrans
  Attempting unin

In [4]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


esgtokenizer = AutoTokenizer.from_pretrained("nbroad/ESG-BERT") ## tokenizer for ESG-BERT  Fine tuning 된 BERT 모델을 ESG카테고리를 분류하는데 사용

esgmodel = AutoModelForSequenceClassification.from_pretrained("nbroad/ESG-BERT")


sentimenttokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert") ## tokenier for financial data Fine tuning 된 BERT 모델을 긍/부정 분류하는뎅데 사용

sentimentmodel  = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

c:\Junho\envs\sudal\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\junho\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Junho\envs\sudal\lib\site-packages\torch\_utils.py:147: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this vers

### 2. 웹크롤링 함수


In [7]:
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)
#크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)       
def makeUrl(search,start_pg,end_pg):
    start_page = makePgNum(start_pg)
    urls=[]
    if start_pg==end_pg:
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        urls.append(url)
        return urls
    else:
        
        for i in range(start_page,end_pg+1):
            page=makePgNum(i)
            url= "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        return urls #여기까지 하면 해당 검색어 n페이지 까지 나오는것 같다.

def articles_crawler(url):  
    url_naver=[] 
    if len(url)==1:
        original_html=requests.get(url[0],headers=headers)
        html=BeautifulSoup(original_html.text,"html.parser")
        url_naver.append(html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info"))
        return url_naver
    else:
        for l in url:
            print(l)
            original_html=requests.get(l,headers=headers)
            html=BeautifulSoup(original_html.text,"html.parser")
            url_naver.append(html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info"))
        return url_naver

In [8]:
def getapi(search,start,end):
    from urllib import parse
    search=parse.quote(search)
    print(search)
    url=makeUrl(search,start,end)
    news_titles=[]
    news_url=[]
    news_contents=[]
    news_dates=[]

    url_naver=articles_crawler(url)
    final_url=[]

    for t in url_naver:
        
        for link in t:
            if "news.naver.com" in link.attrs['href']:
                final_url.append(link.attrs['href'])
            else:
                pass
    for i in tqdm(final_url):
        #각 기사 html get하기
        news = requests.get(i,headers=headers)
        news_html = BeautifulSoup(news.text,"html.parser")
        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')
        content=content.replace(u'\xa0',u'')

        news_titles.append(title)
        news_contents.append(content)

        try:
            html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
            news_date = html_date.attrs['data-date-time']
        except AttributeError:
            news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
            news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
        # 날짜 가져오기
        news_dates.append(news_date)
    return news_titles,news_dates,final_url,news_contents


In [9]:
titles,dates,urls,contents=getapi('삼성SDI',1,1)


%EC%82%BC%EC%84%B1SDI


100%|██████████| 5/5 [00:00<00:00,  5.36it/s]


### 위와 같은 방식으로 뉴스크롤링을 진행하였고, 아래에는 크롤링 된 뉴스를 바탕으로, ESG/감성 분류하는 상황에 대한 코드입니다. 

### 3. 감성분류
- Fine tuning 된 BERT 모델을 ESG카테고리를 분류하는데 사용
- Fine tuning 된 BERT 모델을 긍/부정을 분류하는데 사용

In [10]:
esgdic={
 0:'Business_Ethics',
 1:'Data_Security',
 2:'Access_And_Affordability',
 3:'Business_Model_Resilience',
 4:'Competitive_Behavior',
 5:'Critical_Incident_Risk_Management',
 6:'Customer_Welfare',
 7:'Director_Removal',
 8:'Employee_Engagement_Inclusion_And_Diversity',
 9:'Employee_Health_And_Safety',
 10:'Human_Rights_And_Community_Relations',
 11:'Labor_Practices',
 12:'Management_Of_Legal_And_Regulatory_Framework',
 13:'Physical_Impacts_Of_Climate_Change',
 14:'Product_Quality_And_Safety',
 15:'Product_Design_And_Lifecycle_Management',
 16:'Selling_Practices_And_Product_Labeling',
 17:'Supply_Chain_Management',
 18:'Systemic_Risk_Management',
 19:'Waste_And_Hazardous_Materials_Management',
 20:'Water_And_Wastewater_Management',
 21:'Air_Quality',
 22:'Customer_Privacy',
 23:'Ecological_Impacts',
 24:'Energy_Management',
 25:'GHG_Emissions'}

In [11]:
sentimentdic={
1:'neutral',
2: 'positive',
0: 'negative'
}

In [75]:
from googletrans import Translator

# Translator 클래스 객체 선언 (translator라는 변수명은 마음대로 정해주면 됨)
translator = Translator()

In [83]:
esginfo=[]
sentiment=[]

In [84]:
import pandas as pd
import re
##삼성SDI.csv 파일을 읽는 코드를 작성해줘
sdi=pd.read_csv("에코프로비엠.csv")

In [85]:
content

'“양극재 CAPA 증설 효과 등 높은 외형 성장”반면 투자 의견 ‘매도’로 하향한 보고서도 에코프로비엠 포항사업장. 사진 제공 = 연합뉴스에코프로비엠의 주가가 지난 4월 한 달 사이에만 18% 넘게 오른 가운데 목표주가를 훌쩍 올린 30만원 선으로 제시한 보고서들이 나오면서 눈길을 끌고 있다.4일 증권가에 따르면 지난 2일부터 전날까지 이틀 사이에만 에코프로비엠의 목표주가를 상향 조정한 보고서는 삼성증권, 신한투자증권, 키움증권 등 7개로 집계됐다. 이 기간 목표가를 높인 보고서가 총 62개인 점을 감안하면 상당한 숫자다.지난 2일 에코프로비엠은 연결 기준 올해 1분기 영업이익은 1073억원으로 전년동기대비 161.4% 증가한 것으로 잠정 집계됐다고 공시했다. 매출은 2조110억원으로 전년동기대비 203.6% 증가했다. 순이익 역시 164.4% 늘어난 804억원으로 집계됐다.컨센서스에 근접한 실적을 거둬들이면서 증권가에서도 눈높이를 올리고 있는 것으로 풀이된다.이 중 권준수 키움증권 연구원은 ‘여전히 견조한 전기차(EV) 전방 수요’이라는 제목의 보고서를 통해 에코프로비엠의 목표주가를 34만으로, 투자의견은 ‘매수’로 제시했다.미국의 인플레이션 감축법(IRA) 시행령 발표 이후 에코프로비엠의 원재료 소싱 역량은 한층 부각되고 있다는 평가다.권 연구원은 “이번 실적 발표를 통해 ‘핵심 광물(Criica Mineras)’ 요건 충족을 위해 리튬·니켈 조달 세부 방안을 공유했으며 이를 위해 계열사인 에코프로머티리얼즈(전구체), 에코프로이노베이션(수산화리튬), 에코프로CNG(리사이클링)의 역할이 중요해질 것이다”라고 말했다.그러면서 “올해 원재료 가격 하향 안정화가 예상되는 가운데, 전방 고객사들(BMW, 포드, 현대차·기아 등)의 신차 출시와 양극재 생산능력(CAPA) 증설 효과(2022년 9만5000톤→2023년 18만톤)가 더해지며 높은 외형 성장이 예상된다”고 덧붙였다.키움증권 외에도 한화투자증권(14만원→31만원), 신영증권(15만7500원→30만원), D

In [86]:
import torch
import time
for idx,content in enumerate(sdi['Contents']):
    content = re.sub(pattern="[&gt&lt\][\n一-龥■▲]", repl='', string=content)
    if len(content)>5000:
        esginfo.append(esg)
        sentiment.append(sent)
        continue
    if content is None:
        esginfo.append(esg)
        sentiment.append(sent)    
        continue
    if len(content)<5:
        esginfo.append(esg)
        sentiment.append(sent)
        continue
   
    if content =='':
        esginfo.append(None)
        sentiment.append(None)
        continue

    try: 
        res1= translator.translate(content,src='ko', dest='en').text ##한영 번역

    except:       
        esginfo.append(None)
        sentiment.append(None)
        pass
    esginput = esgtokenizer(res1, return_tensors="pt",truncation=True,max_length=512, add_special_tokens = True) ## 토크나이저 사용
    sentimentinput=sentimenttokenizer(res1, return_tensors="pt",truncation=True,max_length=512, add_special_tokens = True)
    esg=esgdic[torch.argmax(esgmodel(**esginput)['logits']).item()]
    sent=torch.argmax(sentimentmodel(**sentimentinput)['logits']).item()
    esginfo.append(esg)
    sentiment.append(sent)
    print(esg,end=" ")
    print(sent)
    

Systemic_Risk_Management 1
Competitive_Behavior 1
Systemic_Risk_Management 0
Systemic_Risk_Management 0
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Management_Of_Legal_And_Regulatory_Framework 0
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 0
Systemic_Risk_Management 1
Product_Design_And_Lifecycle_Management 1
Business_Model_Resilience 0
Business_Model_Resilience 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 2
Systemic_Risk_Management 0
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 0
Labor_Practices 2
Supply_Chain_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Systemic_Risk_Management 1
Syste

In [87]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from matplotlib import dates as mdates
from bs4 import BeautifulSoup as bs

In [93]:
res = [i for i in range(len(esginfo)) if esginfo[i] == None]

In [96]:
len(esginfo)

1243

In [102]:
len(sentiment)

1243

In [103]:
esginfo = [i for i in esginfo if i is not None]
sentiment = [i for i in sentiment if i is not None]
 

In [104]:
len(sentiment)

1224

In [110]:
esg_df=pd.DataFrame(esginfo,columns=['ESG'])
sentiment_df=pd.DataFrame(sentiment,columns=['Sentiment'])

In [106]:
company=pd.concat([sdi,esg_df],axis=1)

In [116]:
company=pd.concat([company,sentiment_df],axis=1)

In [117]:
company

,Title,Date,URL,Contents,ESG,Sentiment
0,[오후 시황] 2500선 사수하는 코스피…외국인·기관 순매수,2023-05-09 14:24:01,https://n.news.naver.com/mnews/article/011/000...,[\n\n\n\n\n서울 중구 하나은행 딜링룸에서 딜러가 업무를 보고 있다. 연합뉴...,Systemic_Risk_Management,1
1,"보고서 내자 ""공매도 앞잡이냐"" 항의 폭주…욕받이 된 애널들 [돈앤톡]",2023-05-09 12:26:07,https://n.news.naver.com/mnews/article/015/000...,"[\n'매수'는 ""소신 없다"" vs '매도'는 ""공매도 앞잡이""투자의견 낮추자 작성...",Competitive_Behavior,1
2,4월 광주·전남 주식거래대금↑…시가총액 6216억 ↑,2023-05-09 11:29:18,https://n.news.naver.com/mnews/article/003/001...,[\n기사내용 요약한국거래소 4월 광주전남 증시 동향 발표\n\n\n\n[서울=뉴시...,Systemic_Risk_Management,0
3,"[주식 초고수는 지금] 실리콘투, 1분기 호실적 소식에 매수 1위",2023-05-09 11:27:01,https://n.news.naver.com/mnews/article/011/000...,"[\n[미래에셋증권 엠클럽 집계]에스엠, 금양, 두산에너빌리티 등 매수 상위권\n\...",Systemic_Risk_Management,0
4,"코스닥, 외인 나흘째 '팔자'에 830선 밑으로",2023-05-09 11:01:05,https://n.news.naver.com/mnews/article/018/000...,"[\n전거래일보다 1.24% 하락한 831.85외인 1206억, 기관 72억 순매도...",Systemic_Risk_Management,1
...,...,...,...,...,...,...
1219,"2차 전지株, 전기차 기대감 업고 동반 강세…에코프로비엠·엘앤에프 ↑",2021-12-08 17:07:04,https://n.news.naver.com/mnews/article/024/000...,[\nLG엔솔 공모 절차 돌입에 애플·인텔 자율주행차 이슈로 급등글로벌 투자은행 U...,Systemic_Risk_Management,0
1220,에코프로비엠 시총 11.7조…1위 셀트리온헬스케어 바짝 추격,2021-12-08 14:39:01,https://n.news.naver.com/mnews/article/366/000...,[\n\t\t\t 8일 2차전지 업체 에코프로비엠(247540) 주가가...,Systemic_Risk_Management,0
1221,"[개미톡톡] 외국인 사랑 받았던 에코프로비엠 주가하락, 저가 매수 기회?",2021-12-06 17:54:01,https://n.news.naver.com/mnews/article/138/000...,[\n\n\n\n\n주식시장에서 IT종목들은 시대 변화에 빠르게 반응하는 종목들입니...,Systemic_Risk_Management,1
1222,에코프로비엠·엘앤에프 등 2차전지 관련주 급락,2021-12-06 11:07:03,https://n.news.naver.com/mnews/article/366/000...,[\n\t\t\t 6일 장 초반 에코프로비엠(247540)과 엘앤에프(...,Systemic_Risk_Management,1


In [220]:
company["new date"] = pd.to_datetime(company["Date"]).dt.strftime("%Y-%m-%d")

In [126]:
company=company.drop(['Date'], axis=1)

In [127]:
company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      1224 non-null   object
 1   URL        1224 non-null   object
 2   Contents   1224 non-null   object
 3   ESG        1224 non-null   object
 4   Sentiment  1224 non-null   int64 
 5   new date   1224 non-null   object
dtypes: int64(1), object(5)
memory usage: 57.5+ KB


In [129]:
esginfo=company['ESG']
sentimentinfo=company['Sentiment']
dateinfo=company['new date']

In [130]:
stock=pd.read_csv("에코프로비엠_주가.csv")

In [160]:
stock["날짜"] = pd.to_datetime(stock["날짜"]).dt.strftime("%Y-%m-%d")

0       2023-05-09
1       2023-05-09
2       2023-05-09
3       2023-05-09
4       2023-05-09
           ...    
1219    2021-12-08
1220    2021-12-08
1221    2021-12-06
1222    2021-12-06
1223    2021-12-06
Name: new date, Length: 1224, dtype: object

In [212]:
stocksinfo=[]

In [208]:
len(stock[stock['날짜']==dateinfo[300]]['종가'].values)

0

In [213]:
cur_price=stock[stock['날짜']==dateinfo[0]]['종가']
for i in range(len(dateinfo)):
    
    a=stock[stock['날짜']==dateinfo[i]]['종가']
    if len(a)!=0:
        cur_price=a
    stocksinfo.append(cur_price.values[0])
    

In [214]:
stocksinfo

[236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 236500.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 242000.0,
 250000.0,
 250000.0,
 250000.0,
 250000.0,
 250000.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,
 249500.0,

In [231]:
company

,ESG,Sentiment,Stock price,new date
0,Systemic_Risk_Management,1,236500.0,2023-05-09
1,Competitive_Behavior,1,236500.0,2023-05-09
2,Systemic_Risk_Management,0,236500.0,2023-05-09
3,Systemic_Risk_Management,0,236500.0,2023-05-09
4,Systemic_Risk_Management,1,236500.0,2023-05-09
...,...,...,...,...
1219,Systemic_Risk_Management,0,535000.0,2021-12-08
1220,Systemic_Risk_Management,0,535000.0,2021-12-08
1221,Systemic_Risk_Management,1,504100.0,2021-12-06
1222,Systemic_Risk_Management,1,504100.0,2021-12-06


In [224]:
esg_df=pd.DataFrame(esginfo,columns=['ESG'])
sentiment_df=pd.DataFrame(sentiment,columns=['Sentiment'])
stocks_df=pd.DataFrame(stocksinfo,columns=['Stock price'])
dateinfo_df=dateinfo

In [226]:
dateinfo_df=pd.DataFrame(dateinfo)

In [228]:
company=pd.concat([esg_df,sentiment_df,stocks_df,dateinfo_df],axis=1)

In [2]:
## test set. 
import pandas as pd
company=pd.read_csv("esgfile.csv")

In [15]:
stocks=pd.read_csv("에코프로비엠_주가.csv")

In [16]:
stocks

,날짜,종가,전일비,시가,고가,저가,거래량
0,2023.05.09,236500.0,5500.0,239000.0,243500.0,227500.0,1946238.0
1,2023.05.08,242000.0,8000.0,252000.0,253000.0,241500.0,1072455.0
2,2023.05.04,250000.0,500.0,246000.0,254000.0,244000.0,1110726.0
3,2023.05.03,249500.0,17500.0,262500.0,263000.0,247500.0,1559531.0
4,2023.05.02,267000.0,0.0,268500.0,276500.0,259500.0,1515819.0
...,...,...,...,...,...,...,...
1505,2019.03.11,69200.0,700.0,69600.0,70300.0,66100.0,394610.0
1506,2019.03.08,69900.0,100.0,70000.0,70800.0,67600.0,472140.0
1507,2019.03.07,70000.0,1300.0,70300.0,72400.0,69200.0,1019641.0
1508,2019.03.06,68700.0,5500.0,64400.0,69600.0,63700.0,1484460.0


In [13]:
company

,Unnamed: 0,ESG,Sentiment,Stock price,new date
0,0,Systemic_Risk_Management,1,236500.0,2023-05-09
1,1,Competitive_Behavior,1,236500.0,2023-05-09
2,2,Systemic_Risk_Management,0,236500.0,2023-05-09
3,3,Systemic_Risk_Management,0,236500.0,2023-05-09
4,4,Systemic_Risk_Management,1,236500.0,2023-05-09
...,...,...,...,...,...
1219,1219,Systemic_Risk_Management,0,535000.0,2021-12-08
1220,1220,Systemic_Risk_Management,0,535000.0,2021-12-08
1221,1221,Systemic_Risk_Management,1,504100.0,2021-12-06
1222,1222,Systemic_Risk_Management,1,504100.0,2021-12-06


## 30일 기준으로  100일 후의 주가(long term) 예측한다고 할때 (기간에 대한 설정은 자유롭게 , 임의적으로 가능  )

5월 9일

[Epos, Spos, Gpos, Eneg, Sneg,Gneg]
[1,0,0,1,0,0] 과 같은 방식으로 one hot encoding


2021- 12-6~ 5월 9일까지

500일 정도 된다고 하면, 500* 52-> x_train



2021-12-06~ 5d월 9일까지 
2022년 1월 1일 ~ 2022년 4월 1일까지 100* 52->  []
1월 1 4월 1일까지 4월 2일 [3%]
1월 2 4월 2일까지 4월 3일 [4%]





### Reatain 모델 구현

In [277]:
import pandas as pd

X_info=pd.read_csv("X.csv")


In [278]:
X_info.head()

,new date,positive_Business_Ethics,positive_Data_Security,positive_Access_And_Affordability,positive_Business_Model_Resilience,positive_Competitive_Behavior,positive_Critical_Incident_Risk_Management,positive_Customer_Welfare,positive_Director_Removal,positive_Employee_Engagement_Inclusion_And_Diversity,...,negative_Selling_Practices_And_Product_Labeling,negative_Supply_Chain_Management,negative_Systemic_Risk_Management,negative_Waste_And_Hazardous_Materials_Management,negative_Water_And_Wastewater_Management,negative_Air_Quality,negative_Customer_Privacy,negative_Ecological_Impacts,negative_Energy_Management,negative_GHG_Emissions
0,2021-12-06,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-12-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-12-08,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2021-12-09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-12-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [279]:
new_info=X_info.drop('new date',axis=1)

In [280]:
X_train=new_info.to_numpy()

In [281]:
seq_len=30
a=[]
for i in range(31,421):
    temp=X_train[:seq_len,:][::-1]
    a.append(temp)

In [282]:
import numpy as np
X=np.array(a)

In [283]:
y_info=pd.read_csv('yys.csv')

In [284]:
y=y_info['after100'][31:421]

In [329]:
import torch.nn as nn
import torch
class RETAIN(nn.Module):
    def __init__(self,input_dim,embedding_dim, hidden_dim) -> None:
        super().__init__()
        ## 1. 기본적인 parameter 선언
        self.hidden_dim=hidden_dim ## diemension of hidden state, i.e) 128,256, ... 
        self.input_dim=input_dim ## dimension of input time series. 기간을 얼마나 잡을 것인지에 대한 정보를 담음. 
        self.embedding_dim=embedding_dim 
        ## embedded dimension. 단순히 binary encoding의 경우 정보를 많이 저장할 수 없을 수 잇으므로, 정보를 조금더 다채롭게 표현하기 위해서(변수들간의 관계를 고려하기 위해) embedding dim 정의 후  embedding 사용.


        ## 2. 임베딩
        self.embedding=nn.Linear(input_dim,embedding_dim)  ## 'Linearly' embedding imput dimension.
        

        ## 3. 양방향으로 RNN 기반의 모델 부르기 
        self.GRUalpha=nn.GRU(embedding_dim,hidden_dim,batch_first=True) ## GRU, for obtaining alphas
        self.GRUbeta=nn.GRU(embedding_dim,hidden_dim,batch_first=True) ## GRU, for obtaining, Betas. 
        
        ## 4. 구해진 시점에서의 timestep 기반으로 alpha, beta구하기 
        self.LinearAlpha=nn.Linear(hidden_dim,1,bias=True) ## Linear Alpha, making hidden state alpha for every timestep (timestep* hiddendim) into (timestep * 1)
        self.LinearBeta=nn.Linear(hidden_dim,embedding_dim,bias=True) ## Linear Beta, making hidden state beta for every timestep( timestep*hiddendim) into (timestep *featurenumber)
        
        
        ## 5. softmax와 tanh를 통해서  각시점에서의 attention value를 구하기 위한 장치 마련. 
        self.Alphsoftmax=nn.Softmax(dim=1) ##(size of timestep)
        self.Betatanh=nn.Tanh() ##(size of timestep)
        


        ## 6. 마지막 Linear layer을 통과시켜, 1(긍정)의 확률 과 0(부정)의 확률을 구해줌. 
        self.lastLinear=nn.Linear(embedding_dim,3) ##  마지막에 결과가 긍정(1)일 확률과 0 일 확률을 구해줘야하므로, 2개의 dimension을 출력하도록 하였음. 
        self.lastSoftmax=nn.Softmax(dim=1)  

    def forward(self,x):
        h0 = torch.zeros( 1,x.shape[0],self.hidden_dim)# Initialize hidden state with zeros
        #c0 = torch.zeros(1, x.shape[0], self.hidden_dim) for LSTM, in GRU there is no cell state. 
        
        x=self.embedding(x) ## input 임베딩 


        outalpha, _ = self.GRUalpha(x, (h0))  ## outputsize= timestep* hiddensize

        ## 각각의 timestep에 대하여, 그에 대한 hidden state출력해줌. (1번째 시점에서의 hidden state, 2번째 시점에서의 hidden state... 50번째 시점에서의 hidden state,,) 
        ## 논문에서는 alpha 는 조금더 visit자체에 focus를 둔 attention vector



        outbeta,_=self.GRUbeta(x,(h0)) ## outputsize= timestep* hiddensize

        ## 각각의 timestep에 대하여, 그에 대한 hidden state출력해줌. (1번째 시점에서의 hidden state, 2번째 시점에서의 hidden state... 50번째 시점에서의 hidden state,,) 
        ## 논문에서는 beta는 병들(fetaure)들에 초점을 맞춘 attention vector. 


        attentionalpha=self.LinearAlpha(outalpha) ##timestep *hiddensize-> tiemstep* 1
        attentionbeta=self.LinearBeta(outbeta) ##timestep *hiddensize-> tiemstep* embedding_dim 
        


        attentionalpha=self.Alphsoftmax(attentionalpha)  ##dimension:softmax통과시키는 이유? 가중치를 정규화(normalize에서 )해서 표현하기 우해
        attentionbeta=self.Betatanh(attentionbeta) ##dimension : tanh를 사용하는 이유?  softmax와는 다른 어떤 의미를 capture하기 위해서. 

        

        
        alphabeta=torch.mul(attentionalpha[:,:],attentionbeta[:,:]) ## dimension: timestep. 
        totalsum=torch.sum(alphabeta*x,dim=1) ## sum up attention value to get the probability ,.

        #print(totalsum.shape)
        a=self.lastLinear(totalsum)
        
        b=self.lastSoftmax(a)
        
        return attentionalpha,attentionbeta,b

In [330]:
model=RETAIN(52,200,128)

In [287]:
X.shape

(390, 30, 52)

In [288]:
X=np.array(X,dtype=np.float64)

In [289]:
y=y.to_numpy()

In [290]:
for i in range(len(y)):
    if y[i]>5:
        y[i]=2
    elif y[i]<-5:
        y[i]=1
    else:
        y[i]=0

In [291]:
import pandas as pd
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None, target_transform=None):
        self.labels = y
        self.X=X
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        t_series = self.X[idx,:,:]
        label = self.labels[idx]
        return t_series, label

In [292]:
from torch.utils.data import DataLoader
train_data=CustomDataset(X[:300],y[:300])
test_Data=CustomDataset(X[300:],y[300:])


In [293]:
y.shape

(390,)

In [294]:
len(train_data)

300

In [295]:
train=DataLoader(train_data,batch_size=5,shuffle=True)
test=DataLoader(test_Data,batch_size=1,shuffle=True)

In [331]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

## 모델 훈련시키기

In [332]:
epochs=100
for epoch in range(epochs):
    for batch_idx, samples in enumerate(train):
        size=len(train)
        x_train,y_train=samples
        _,_,prediction=model(x_train)
        loss = loss_fn(prediction, y_train.long())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #loss, current = loss.item(), (batch_idx + 1) * len(X)
        if batch_idx % 5 == 0:
            loss, current = loss.item(), (batch_idx + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

loss: 1.094849  [  390/   60]
loss: 1.095883  [ 2340/   60]
loss: 0.844039  [ 4290/   60]
loss: 1.380040  [ 6240/   60]
loss: 0.964685  [ 8190/   60]
loss: 1.128971  [10140/   60]
loss: 0.779970  [12090/   60]
loss: 0.953038  [14040/   60]
loss: 1.484860  [15990/   60]
loss: 0.951513  [17940/   60]
loss: 1.118656  [19890/   60]
loss: 1.123797  [21840/   60]
loss: 1.126380  [  390/   60]
loss: 1.318840  [ 2340/   60]
loss: 1.126075  [ 4290/   60]
loss: 0.955608  [ 6240/   60]
loss: 1.128100  [ 8190/   60]
loss: 0.788092  [10140/   60]
loss: 1.126417  [12090/   60]
loss: 1.137488  [14040/   60]
loss: 1.121955  [15990/   60]
loss: 0.779950  [17940/   60]
loss: 0.769158  [19890/   60]
loss: 1.304261  [21840/   60]
loss: 0.773457  [  390/   60]
loss: 0.773408  [ 2340/   60]
loss: 0.770256  [ 4290/   60]
loss: 0.767772  [ 6240/   60]
loss: 0.774323  [ 8190/   60]
loss: 0.767672  [10140/   60]
loss: 0.954038  [12090/   60]
loss: 0.947215  [14040/   60]
loss: 0.958498  [15990/   60]
loss: 1.32

## 훈련된 모델을 통해 test데이터에 적용한 예시

In [319]:
model.eval()

RETAIN(
  (embedding): Linear(in_features=52, out_features=200, bias=True)
  (GRUalpha): GRU(200, 128, batch_first=True)
  (GRUbeta): GRU(200, 128, batch_first=True)
  (LinearAlpha): Linear(in_features=128, out_features=1, bias=True)
  (LinearBeta): Linear(in_features=128, out_features=200, bias=True)
  (Alphsoftmax): Softmax(dim=1)
  (Betatanh): Tanh()
  (lastLinear): Linear(in_features=200, out_features=3, bias=True)
  (lastSoftmax): Softmax(dim=1)
)

In [335]:
alpha,beta,pred=model(torch.Tensor(X[301,:,:].reshape(1,30,52)))

In [337]:
torch.argmax(pred)

tensor(1)

In [341]:
alpha.shape

torch.Size([1, 30, 1])

## 아래와 같은방식으로 현재 시점에서 Positive/Negative/Neutral 한 결과가 나올 확률과, 그확률에 기여한 변수의 영향력을 확인 할 수 있습니다. 

In [347]:
print(pred)

tensor([[2.7899e-04, 9.0302e-01, 9.6700e-02]], grad_fn=<SoftmaxBackward0>)


In [350]:
print("Probability of Positive Result :{}".format(pred[0,0]))
print("Probability of Negative Result :{}".format(pred[0,1]))
print("Probability of Neutral Result :{}".format(pred[0,2]))

Probability of Positive Result :0.00027898585540242493
Probability of Negative Result :0.9030212759971619
Probability of Neutral Result :0.09669972211122513


### 이렇듯, Negative Risk 가 큰 경우, 모든 투자자들한테 워닝, 

In [352]:
for i in range(30):
    alpha[:,i,:]
    valueimpact=(beta[:,i,:])
    for k in range(52):
        if X[301,i,k]==0:
            continue
        weight=(model.embedding.weight[:,k])
        weight*valueimpact
        ans=model.lastLinear(weight*valueimpact)*alpha[:,i,:]
        print(ans.shape)
        print("effect of timestep {} and feature {} for obtaining Negative Result:  {}".format(i+1,k+1,ans[:,1]))

torch.Size([1, 3])
effect of timestep 14 and feature 39 for obtaining Negative Result:  tensor([3.5345e-05], grad_fn=<SelectBackward0>)
torch.Size([1, 3])
effect of timestep 26 and feature 51 for obtaining Negative Result:  tensor([2.7516e-05], grad_fn=<SelectBackward0>)
torch.Size([1, 3])
effect of timestep 27 and feature 42 for obtaining Negative Result:  tensor([2.4699e-05], grad_fn=<SelectBackward0>)
torch.Size([1, 3])
effect of timestep 28 and feature 45 for obtaining Negative Result:  tensor([3.0719e-05], grad_fn=<SelectBackward0>)
torch.Size([1, 3])
effect of timestep 30 and feature 42 for obtaining Negative Result:  tensor([2.4856e-05], grad_fn=<SelectBackward0>)


14일차의 negative_Management_Of_Legal_And_Regulatory_Framework  가 상대적으로 주식이 떨어지는 것에 영향을 줄 수 있음을 확인 할 수 있음 